In [1]:
import json
import ipywidgets as widgets
import os
import time
import numpy as np
import random


In [2]:
os.getcwd()

'C:\\Users\\jfwal\\OneDrive\\Documents\\GitHub\\cyberAR\\_Code Device\\AR Labs\\Assets\\Resources\\jsonDefinitions'

In [3]:



# set up the helper routines to load the jsons
# written by J Wallin

def jsonFromFile(fname):
    f = open(fname,"r")
    s = f.read()
    f.close()
    return json.loads(s)

def jsonToFile(fname):
    print(fname)
    
def listFiles(assetPath, printFiles):
    finalList = []
    fullList = os.listdir(assetPath)
    for f in fullList:
        if f.find("json") == len(f)-4:
            print(f)
            finalList.append(f)
    return finalList

def loadFileIntoDictionary(path, flist):
    jdata = {}
    for f in flist:
        jdata[f] = jsonFromFile(path + f)
    return jdata



def writeScene(myScene, verbose):
    # save it to an output file
    outputFile = myScene["jsonFileName"]

    print(os.path.isfile(outputPath + outputFile))

    # if there is a file already there
    print(outputPath + outputFile)
    fileExists = os.path.isfile(outputPath + outputFile)

    if verbose:
        while (fileExists or outputFile == "" ):
    
            print("This is a listing of files in this path: ")
            flist = os.listdir(outputPath)
            for f in flist:
                print(f)
            yn = input("The file " + outputPath + outputFile +" already exists \n Do you want to overwrite it? ")
            if yn == "y" or yn == "Y":
                fileExists = False
            else:
                outputFile = input("What is the new name for our file? ")
                fileExists = os.path.isfile(outputPath + outputFile)
                

    print("\nWriting " + outputPath+outputFile + "\n\n")


    # form the final scene as a string
    myScene["jsonFileName"] = outputFile
    finalJson = json.dumps(myScene, indent=4)
    f = open(outputPath + outputFile, "w", encoding='utf-8')
    f.write(json.dumps(myScene, ensure_ascii=False, indent=4))
    f.close()

    
def createScene(moduleDescription, sceneData, olist, verbose=False):
    sceneData["objects"] = olist
    
    
    # make a copy of the generic scene for the activity file
    myScene = jdata['genericActivity.json'].copy()

    # apply the module description data ot the sceneObject
    myScene.update(moduleDescription)
    myScene.update(sceneData)


    # we can dump the scene to see what it looks like so far  
    if verbose:
        print("This is the Scene\n\n\n")
        print(json.dumps(myScene, indent=4))    
    
    writeScene(myScene, verbose)    
    
    
# this is a code stub for merging two activity files together
def concatenateScenes( outputPath, outputFile, sceneList):

    print(outputPath, outputFile)
    # open the new file
    f = open(outputPath + outputFile, 'w', encoding='utf-8')
    
    for i in range(len(sceneList)):
        print("  \n scene")
        print(i)
        currentScene = sceneList[i]
        print(currentScene)
        
        # read in the files
        fn1 = outputPath + currentScene
        s1 = jsonFromFile(fn1)

        # Dump the file - but do NOT use any indents.  This will flatten
        # the file into a single line.  We then write it out as a line with a newline character.
        a1 = json.dumps(s1, ensure_ascii=False)
        f.write(a1 + "\n")

    # close it
    f.close()
    
def findKeyDiffs(baseClass, olist):
    # find the differences between two dictionaries
    
    # get the keys for the base class
    baseKeys = list(baseClass.keys())
    
    # find the set of common keys for the objects and the set of unique keys
    commonKeys = []
    uniqueKeys = []
    
    # get the key for each object
    for ii in range(len(olist)):
        cc = olist[i]
        objectKeys = list(cc.keys())
    
        # the union of the keys
        uKeys = list(set(baseKeys + objectKeys))
        
        # the intersection of keys
        iKeys = list(set(baseKeys).intersection(objectKeys))
        
        # find the elements only in one list or the other
        aKeys = list( set(baseKeys)^set(objectKeys))

        commonKeys = commonKeys + uKeys
        uniqueKeys = uniqueKeys + aKeys
        
    
    commonKeys = list(set(commonKeys))
    uniqueKeys = list(set(uniqueKeys))
    
    return commonKeys, uniqueKeys
    
        #for ii in range(len(k1)):
        #    k = k1[ii]
        #    if o1[k] != o2[k]:
        #        print(k)
    
    #


def findDiffs(commonKeys, baseClass, olist):
    
    # for each key
    for c in commonKeys:
        baseValue = baseClass[c]
        
        # are there any differences
        diffs = False
        for i in range(len(olist)):
            newValue = olist[i][c]
            if newValue != baseValue:
                diffs = True


        if diffs:
            print("{0:>15}".format(c), end="")
            for i in range(len(olist)):

                if baseValue != olist[i][c]:

                    print("  X   ", end="")
                else:
                    print("  .   ", end="")

            print()
    print(len(olist))
    
#commonKeys, uniqueKeys = findKeyDiffs(earthObject, sortList)   
#findDiffs(commonKeys, earthObject, sortList)


def createTextLine( textObjectName, theText, fSize, lineNumber, topPosition, rotationAngle, textColor, offSet, parentObject, labelScale):
    
    tdata = {
        "textField": "The Earth and Moon",
        "color": textColor,
        "fontSize": fSize,
        "wrapText": False
        }



    position1 = {"x" :topPosition['x'] + lineNumber * offSet['x'],
                "y": topPosition['y'] + lineNumber * offSet['y'],
                "z":topPosition['z'] + lineNumber * offSet['z']
               }

    tdata["textField"] = theText
    textObject = {
        "type": "Prefabs/textPrefab",
        "tmp": json.dumps(tdata),
        "position": position1.copy(),
        "rotation": rotationAngle,
        "scale": {"x": labelScale, "y": labelScale, "z": labelScale},
        "enabled": True,
        "parentName": parentObject,
        "name": textObjectName
    }
    
    return textObject.copy()



def createObjectModification(
    name, # object name
    activationCondition, 
    reactivateObject=False, 
    parentObject="[DYNAMIC]", 
    audioClipName = "",
    goCallback ="",
    newColor= None,
    enable=True, 
    newPosition=None, 
    newEulerAngles=None, 
    newScale=None,
    tmp=None):
    
    
    mm = {"name": name,
         "parentObject": parentObject,
         "activationConditions" : activationCondition,
         "reactiveObject": reactivateObject,
          "enable": enable,
          "position": {"x":0.0, "y":0.0, "z":0.0},
          "newPosition": False,
          "eulerAngles": {"x":0.0, "y":0.0, "z":0.0},
          "newEulerAngles": False,
          "scale": {"x":0.0, "y":0.0, "z":0.0},
          "newScale": False,
             }
                        
    if newPosition != None:
        mm["position"] = newPosition
        mm["newPosition"] = True
            
    if newEulerAngles != None:
        mm["eulerAngles"] = newEulerAngles
        mm["newEulerAngles"] = True
            
    if newScale != None:
        mm["scale"] = newScale
        mm["newScale"] = True
            
    if newColor != None:
        mm['color'] = newColor
    
    if tmp != None:
        mm['tmp'] = tmp
        
    return mm.copy()


def createButton(name, 
                 label, 
                 position, 
                 labelOffset = 0.1,
                 scale = 1, 
                 parentObject = "[_DYNAMIC]",
                 color=[255,0,0,255], 
                 eulerAngles= {"x":90, "y":180.0, "z":0.0}):

    
    scriptDataA = {
    "name": "demoButtonActions",
    "callBackObjects": ["demoModule"]
    }
    scriptDataA = json.dumps(scriptDataA) 


    brbData = {
                "type": "Prefabs/demoButtonPrefab",
                "parentName": parentObject,
                "position": position,
                "eulerAngles": eulerAngles,
                "scale": {"x":scale, "y":scale, "z":scale},
                "name": name, 
                "color": color,
                "componentsToAdd": [scriptDataA]
            }
    
    labelPosition  = {"x":0.0, "y":0.0, "z":labelOffset}
    tdata = {
        "textField": "",
        "color": [255,255,0,255],
        "fontSize": 1.0,
        "wrapText": False
        }
    tdata["textField"] = label
    labelScale = 0.6 * scale
    brbLabel = {
        "type": "Prefabs/textPrefab",
        "tmp": json.dumps(tdata),
        "position": labelPosition,
        "eulerAngles": eulerAngles,
        "scale": {"x": labelScale, "y": labelScale, "z": labelScale},
        "enabled": True,
        "parentName": name,
        "name": name+"Label"  
    }
    return brbData, brbLabel


In [4]:
    
# define the path to access the generic templates and the output files
assetPath = "C:/Users/jfwal/OneDrive/Documents/GitHub/cyberAR/\_Code Device/AR Labs/Assets/Resources/jsonDefinitions/genericDefinitions/"     
outputPath = "C:/Users/jfwal/OneDrive/Documents/GitHub/cyberAR/\_Code Device/AR Labs/Assets/Resources/jsonDefinitions/"  
assetPath = "genericDefinitions/"
outputPath = "./"


# get the list of all the files in the generic directory and load them into a dictionary
flist = listFiles(assetPath,  True)
jdata = loadFileIntoDictionary(assetPath, flist)


# this prints out the assets so you can review them
showFiles = False
if (showFiles):
    for f in flist:
        print("===== file name" + f)
        print("---------\n",f, "\n\n\n", json.dumps(jdata[f], indent=4))
        
#print(json.dumps(jdata["genericActivity.json"]))



genericActivity.json
genericObject.json
genericPointerReceiver.json
genericRigidBody.json
genericScriptSimpleRotation.json


In [5]:
# variables for the objects


timeRate = 1.0
basketballRotationTime = 2
basketballSize = 0.3




############ 


# this uses a simple prefab
sun = {
        "type": "Prefabs/SunPrefab",
        "position": {"x":-39.0, "y":0, "z":0},
        "scale": {"x":5, "y":5, "z":5},
        "name": "Sun",
        "componentsToAdd": []
    }

# make a copy of the generic dictionary for objects    
sunObject = jdata['genericObject.json'].copy()
sunObject.update(sun)



############
scriptData1 = {
    "name": "simpleRotation",
    "timeRate": timeRate,
    "rotationTime": basketballRotationTime
}
script1 = json.dumps(scriptData1) 

scriptData2 = {
    "name": "checkAngle",
    "audioClipSuccess": None,
    "azmTarget": 0,
    "azmTolerance": 30,
    "altTarget": 0,
    "altTolerance": 30,
    "callBackObjects": ["demoModule"]

}
script2 = json.dumps(scriptData2) 

############
############    
componentsToModify= {
                        "RigidBody": 
                        {
                            "drag": 1.0,
                            "isKinematic": True,
                            "useGravity": False
                        },
                        "PointerReceiver": 
                        {
                        }
                    }


basketballPosition = { "x":0.0, 
                 "y": 0.0, 
                 "z":2.5}
basketballScale = {"x": basketballSize,
            "y": basketballSize,
            "z": basketballSize }

basketball = {
                "type": "Prefabs/moveableSphere",
                "position": basketballPosition,
                "scale": basketballScale,
                "name": "basketball", 
                "parent": "[_DYNAMIC]",
                "texture": "Textures/balldimpled",
                "componentsToAdd": [script1, script2]
            }


# make a copy of the generic dictionary for objects
basketballObject = jdata['genericObject.json'].copy()

# add the generic component attributes
basketballObject.update(jdata['genericRigidBody.json'])
basketballObject.update(jdata['genericPointerReceiver.json'])
basketballObject.update(basketball)

# override the generic component attributes
for c in componentsToModify.keys():
    basketballObject[c].update(componentsToModify[c])
         
        
######

textObjectName = "sectionLabel"
theText = "The Moon Phase Challenge"
fSize=  1.0
lineNumber = 0
topPosition = {"x":0.0, "y": 0.75, "z":2.50}
rotationAngle = {"x": 0.0, "y": 0.0, "z": 0.0}
textColor = [255,255,255,255]
offSet = {"x":0.0, "y":-0.15, "z":0.0}
parentObject = "[_DYNAMIC]"
labelScale = 0.6



sectionLabel = createTextLine( textObjectName, 
               theText, 
               fSize, 
               lineNumber, 
               topPosition, 
               rotationAngle, 
               textColor, 
               offSet, 
               parentObject, 
               labelScale)

####

textObjectName = "taskLabel"
theText = "Introduction"
fSize=  2
lineNumber = 1
labelScale = 0.6



taskLabel = createTextLine( textObjectName, 
               theText, 
               fSize, 
               lineNumber, 
               topPosition, 
               rotationAngle, 
               textColor, 
               offSet, 
               parentObject, 
               labelScale)
 

    
    
'''
tdata = {
        "textField": "The Earth and Moon",
        "color": textColor,
        "fontSize": fSize,
        "wrapText": False
        }
        
        
taskLabelPosition = {"x":1.0, "y":-0.75, "z":0.0}

tdata["textField"] = "Introduction"
labelScale = 1.5
taskLabelBottom = {
        "type": "Prefabs/textPrefab",
        "tmp": json.dumps(tdata),
        "position": taskLabelPosition,
        "eulerAngles": {"x": 0.0, "y": 0.0, "z": 0.0},
        "scale": {"x": labelScale, "y": labelScale, "z": labelScale},
        "enabled": True,
        "parentName": "[_DYNAMIC]",
        "name": "taskLabel"     
    }

        
'''      

############ 
############     



'\ntdata = {\n        "textField": "The Earth and Moon",\n        "color": textColor,\n        "fontSize": fSize,\n        "wrapText": False\n        }\n        \n        \ntaskLabelPosition = {"x":1.0, "y":-0.75, "z":0.0}\n\ntdata["textField"] = "Introduction"\nlabelScale = 1.5\ntaskLabelBottom = {\n        "type": "Prefabs/textPrefab",\n        "tmp": json.dumps(tdata),\n        "position": taskLabelPosition,\n        "eulerAngles": {"x": 0.0, "y": 0.0, "z": 0.0},\n        "scale": {"x": labelScale, "y": labelScale, "z": labelScale},\n        "enabled": True,\n        "parentName": "[_DYNAMIC]",\n        "name": "taskLabel"     \n    }\n\n        \n'

In [6]:
basketballObject

{'name': 'basketball',
 'parentName': '[_DYNAMIC]',
 'type': 'Prefabs/moveableSphere',
 'tag': '',
 'active': True,
 'position': {'x': 0.0, 'y': 0.0, 'z': 2.5},
 'euleriAngles': {'x': 0.0, 'y': 0.0, 'z': 0.0},
 'scale': {'x': 0.3, 'y': 0.3, 'z': 0.3},
 'material': '',
 'texture': 'Textures/balldimpled',
 'textureByURL': '',
 'color': [1.0, 1.0, 1.0, 1.0],
 'transmittable': False,
 'RigidBody': {'mass': 1.0,
  'drag': 1.0,
  'angularDrag': 0.0,
  'isKinematic': True,
  'useGravity': False,
  'xConstraint': False,
  'yConstraint': False,
  'zConstraint': False,
  'xRotationConstraint': True,
  'yRotationConstraint': True,
  'zRotationConstraint': True},
 'PointerReceiver': {'draggable': True,
  'kinematicWhileIdle': True,
  'faceWhileDragging': False,
  'matchWallWhileDragging': False,
  'invertForward': False},
 'componentsToAdd': ['{"name": "simpleRotation", "timeRate": 1.0, "rotationTime": 2}',
  '{"name": "checkAngle", "audioClipSuccess": null, "azmTarget": 0, "azmTolerance": 30, "al

In [7]:


olist = [sun, basketballObject, sectionLabel, taskLabel ] #, sun, earthSizeLabel, moonLabel, moonSizeLabel]

moduleButtonOffset = 1.0

name = "brbNextModule"
label = "Next\n Module"
labelOffset = 0.13
scale= 1.0

parentObject = "sectionLabel"
position = {"x": moduleButtonOffset, "y": 0, "z":0}
o1, o2 =  createButton(name, label, position, labelOffset, scale, parentObject=parentObject)
o1["childColor"] = [0, 255, 255, 255]
o1["childName"] = "button"
olist.append(o1)
olist.append(o2)


name = "brbPreviousModule"
label = "Previous \n Module"
labelOffset = 0.13
scale= 1.0
parentObject = "sectionLabel"
position = {"x": -moduleButtonOffset, "y": 0, "z":0}
o1, o2 =  createButton(name, label, position, labelOffset, scale=scale, parentObject=parentObject)
o1["childColor"] = [0, 255, 255, 255]
o1["childName"] = "button"
olist.append(o1)
olist.append(o2)

##
clipButtonOffset = 0.5
parentObject = "taskLabel"
labelOffset = 0.13


name = "brbNextClip"
label = "Next"
position = {"x": clipButtonOffset, "y": 0, "z":0}
o1, o2 =  createButton(name, label, position, parentObject=parentObject)
o1["childColor"] = [0, 255, 0, 255]
o1["childName"] = "button"
olist.append(o1)
olist.append(o2)


name = "brbPreviousClip"
label = "Previous"
position = {"x": -clipButtonOffset, "y": 0, "z":0}
o1["childColor"] = [0, 255, 0, 255]
o1["childName"] = "button"
o1, o2 = createButton(name, label, position, parentObject=parentObject)
olist.append(o1)
olist.append(o2)




In [8]:
olist

[{'type': 'Prefabs/SunPrefab',
  'position': {'x': -39.0, 'y': 0, 'z': 0},
  'scale': {'x': 5, 'y': 5, 'z': 5},
  'name': 'Sun',
  'componentsToAdd': []},
 {'name': 'basketball',
  'parentName': '[_DYNAMIC]',
  'type': 'Prefabs/moveableSphere',
  'tag': '',
  'active': True,
  'position': {'x': 0.0, 'y': 0.0, 'z': 2.5},
  'euleriAngles': {'x': 0.0, 'y': 0.0, 'z': 0.0},
  'scale': {'x': 0.3, 'y': 0.3, 'z': 0.3},
  'material': '',
  'texture': 'Textures/balldimpled',
  'textureByURL': '',
  'color': [1.0, 1.0, 1.0, 1.0],
  'transmittable': False,
  'RigidBody': {'mass': 1.0,
   'drag': 1.0,
   'angularDrag': 0.0,
   'isKinematic': True,
   'useGravity': False,
   'xConstraint': False,
   'yConstraint': False,
   'zConstraint': False,
   'xRotationConstraint': True,
   'yRotationConstraint': True,
   'zRotationConstraint': True},
  'PointerReceiver': {'draggable': True,
   'kinematicWhileIdle': True,
   'faceWhileDragging': False,
   'matchWallWhileDragging': False,
   'invertForward': Fa

In [9]:


timesList =   [32, 20, 29, 6, 6, 6, 6, 6, 6, 6,  6,  6.5]
audioList =   [0,  1,  2,  3, 4, 5, 6, 7, 8, 9, 10, 11]
autoAdvance = [True,  True,  True,  False,  False,  False,  False,  False,  False,  False,  False,  False]
aztarget =    [  -1,    -1,    -1,      0,     45,     90,    135,    180,    225,    270,    315,     -1]

info = ["Introduction", "Introduction","Introduction", "New Moon", "Waxing Crescent", "First Quarter",
       "Waxing Gibbous", "Full Moon", "Waning Gibbous", "Third Quarter", "Waning Crescent", "Good Job!"]
                            

#for i in range(len(timesList)):
#    print(i, timesList[i], audioList[i], autoAdvance[i], info[i])



    
textColor = [255,255,255,255]
fSize=  1
tdata = {
        "textField": "The Earth and Moon",
        "color": textColor,
        "fontSize": fSize,
        "wrapText": False
        }
textPosition = {"x":0, "y":0.5, "z": 3.1}
###################

clipList = []
for i in range(len(timesList)):
    print(i, timesList[i], audioList[i], autoAdvance[i], info[i])
    
    tdata['fontSize'] = 1
    tdata['textField'] = info[i]
    tdata["parentObject"] =  "[_DYNAMIC]"

    
    # create the change the task label
    oc0a = createObjectModification(
        "taskLabel", 
        0,  
        tmp =json.dumps(tdata)
        )
    
    # change the target location
    oc0b = createObjectModification(
        "basketball", 
        0
        )
    
    if aztarget[i] > -0.01:
        s1 ={"name": "checkAngle", "audioClipSuccess": None, "azmTarget": aztarget[i], "azmTolerance": 30, "altTarget": 0, "altTolerance": 30}
        oc0b["componentsToAdd"] = [json.dumps(s1)]
        

############ button and navigation management
    # this is a placeholder - not currently used
    oc0c = createObjectModification(
        "brbPreviousClip", 
        0 
        )
    

    ###
    oc0d = createObjectModification(
        "brbNextClip", 
        0 
        )    
  

    
    if i != len(timesList) -1:
        oc0d["scale"] = {"x": 1, "y":1, "z":1}
        oc0d["newScale"] = True
    else:
        oc0d["scale"] = {"x": 0.01, "y":0.01, "z":0.01}
        oc0d["newScale"] = True       

    ###
    oc0e = createObjectModification(
        "brbNextModule", 
        0 
        )        
    if i == len(timesList) -1:
        oc0e["childColor"] = [255, 0, 0, 255]
        oc0e["childName"] = "button"
        oc0e["scale"] = {"x": 1, "y":1, "z":1}
        oc0e["newScale"] = True    
    
    if i == 0:
        oc0e["scale"] = {"x": 0.01, "y":0.01, "z":0.01}
        oc0e["newScale"] = True         
    
    clipObj = {"clipName":"clip"+str(i), 
    "audioClipString":"Audio/moonLab/moonLab_2_"+str(i)+"_0",
    "timeToEnd":timesList[i], 
    "autoAdvance": autoAdvance[i],                
    "objectChanges": [oc0a, oc0b, oc0c, oc0d, oc0e] }
    
    

        
    
    #print(json.dumps(clipObj))
    clipList.append(clipObj)
    
    
    



# delete the useless bits at the beginning
del clipList[1]
del clipList[1]
                


clipList2 = clipList[1:len(clipList)-1]
random.shuffle(clipList2)
clipList[1:len(clipList)-1] = clipList2

print(clipList[0]["audioClipString"])
clipList[0]["audioClipString"] ="Audio/moonLab/moonlab_2_20_0"
clipList[0]["timeToEnd"] = 10

print(clipList[0]["audioClipString"])

#for i in range(len(clipList)):
#    print(i, clipList[i]['clipName'])
    
#clipList[0]   
    


0 32 0 True Introduction
1 20 1 True Introduction
2 29 2 True Introduction
3 6 3 False New Moon
4 6 4 False Waxing Crescent
5 6 5 False First Quarter
6 6 6 False Waxing Gibbous
7 6 7 False Full Moon
8 6 8 False Waning Gibbous
9 6 9 False Third Quarter
10 6 10 False Waning Crescent
11 6.5 11 False Good Job!
Audio/moonLab/moonLab_2_0_0
Audio/moonLab/moonlab_2_20_0


In [10]:
import random
random.shuffle(clipList[1:len(clipList)-1])    



In [11]:
a = [1,2,3,4,5]
a[2:3] = a[1:2]
a

[1, 2, 2, 4, 5]

In [12]:
aa = [1,1, 1, 1, 2,3,4,5,6,7,8]
aaa = aa[4:len(aa)]
random.shuffle(aaa)
aa[4:] = aaa

del aa[2]
print(aa)

del aa[2]
print(aa)

del aa[2]
print(aa)


del aa[2]
print(aa)



[1, 1, 1, 2, 5, 3, 4, 7, 8, 6]
[1, 1, 2, 5, 3, 4, 7, 8, 6]
[1, 1, 5, 3, 4, 7, 8, 6]
[1, 1, 3, 4, 7, 8, 6]


In [13]:
# The moduleDescription is the text information that is NOT used by the application at runtime.
# It is only data that we will reference in the database of modules and editing.

moduleDescription = {
        "jsonFileName": "basketballPhases2.json",
        "moduleName": "Basketball Challenge",
        "description": "A student challenge demonstrate their knowledge of moon phases",  
        "author": "John Wallin",
        "authorInstitution": "Middle Tennessee State University",
        "dateCreated": time.strftime("%c")
    }


# MediaInfo[ { "id:", "type": 1}
# MediaURL

# sceneData contains information about prefabs and prefab specific data
sceneData = {
        "prefabName": "demoPrefab",
        "specificName": "Moon Phase Challenge",  # this will appear on the navigation window
        "educationalObjectives": ["Students will learn that phases are caused by the angle between the Moon, Earth, and Sun"],
        "instructions": ["NA"],
    
        # specific to the demo prefab
        "introAudio": "",
        "createObjects": True,
        "destroyObjects": True,
        "restoreLights": True,
        "timeToEnd": 2000,
        "objects": [],
        "useSunlight": True,
        "clips":clipList
}



createScene(moduleDescription, sceneData, olist, verbose=False)

True
./basketballPhases2.json

Writing ./basketballPhases2.json




In [14]:

outputFile = "demo10.json"
#concatenateScenes(outputPath, outputFile, ['basketBallDemo.json', 'miniEarthMoon.json', 'earthMoon.json'])

outputPath = ""
concatenateScenes(outputPath, outputFile, ['basketballPhases2.json'] )

#concatenateScenes(outputPath, outputFile, ['moonEarthIntroduction.json'] )

                  #,'miniEarthMoon.json']  
                  # 'earthMoon.json'])


 demo10.json
  
 scene
0
basketballPhases2.json


In [15]:
outputFile = "demo10.json"
'''
concatenateScenes(outputPath, outputFile, [
    #"moonEarthIntroduction.json",
    'moonEarthSizes.json',
    'synchronousrotation.json',
    'phasesFromEarth.json',
    'phasesOfEarth.json',
    'northpoleview.json',
    'basketballPhases.json',
    'basketballPhases2.json'] )
'''
concatenateScenes(outputPath, outputFile, [
    #"moonEarthIntroduction.json",
    'moonEarthSizes.json'
   # 'synchronousrotation.json',
   # 'phasesFromEarth.json',
   # 'phasesOfEarth.json',
   # 'northpoleview.json',
   # 'basketballPhases.json',
   # 'basketballPhases2.json'
    ] )


 demo10.json
  
 scene
0
moonEarthSizes.json


In [16]:

'''
zStart = 3.0


topPosition1 = {"x": 0.0 ,
                "y": +0.5,
                "z": zStart
               }
rotationAngle = {"x": 0.0,
                "y": 0.0,
                "z": 0.0
               }
textColor = [255, 255, 0, 255]
labelScale =2.0
fSize = 1.0
offSet = {"x": 0.0 ,
            "y": -0.2,
            "z": 0.0
            }
parentObject = "[_DYNAMIC]"

textObjectName = "title"
theText = "The Earth and Moon"
lineNumber = 0

textObjectList = []
to0 = createTextLine( textObjectName, theText, fSize, lineNumber, topPosition1, rotationAngle, textColor, offSet, parentObject, labelScale)
textObjectList.append(to0)



topPosition = {"x": 0.0 ,
                "y": +0.4,
                "z": zStart
              }
fSize = 0.65
textObjectName = "line1"
theText = "Learn about Sizes and Scales"
lineNumber = 1
to1 = createTextLine( textObjectName, theText, fSize, lineNumber, topPosition, rotationAngle, textColor, offSet, parentObject, labelScale)
textObjectList.append(to1)

fSize = 0.65
textObjectName = "line2"
theText = "Determine how the Moon rotates"
lineNumber = 2
to2 = createTextLine( textObjectName, theText, fSize, lineNumber, topPosition, rotationAngle, textColor, offSet, parentObject, labelScale)
textObjectList.append(to2)


fSize = 0.65
textObjectName = "line3"
theText = "Diagram how sunlight illuminates the moon "
lineNumber = 3
to3 = createTextLine( textObjectName, theText, fSize, lineNumber, topPosition, rotationAngle, textColor, offSet, parentObject, labelScale)
textObjectList.append(to3)

fSize = 0.65
textObjectName = "line4"
theText = "Learn the names of moon phases and their order"
lineNumber = 4
to4 = createTextLine( textObjectName, theText, fSize, lineNumber, topPosition, rotationAngle, textColor, offSet, parentObject, labelScale)
textObjectList.append(to4)

fSize = 0.65
textObjectName = "line5"
theText = "Measure orbital periods"
lineNumber = 5
to5 = createTextLine( textObjectName, theText, fSize, lineNumber, topPosition, rotationAngle, textColor, offSet, parentObject, labelScale)
textObjectList.append(to2)


textPosition = { "x":1.5, 
                 "y": 0.0, 
                 "z":zStart}
makerSize = 0.1
markerBall = {
                "type": "Prefabs/tinySphere",
                "parent": parentObject,
                "position": topPosition,
                "scale": {"x":makerSize, "y":makerSize, "z":makerSize},
                "name": "markerBall", 
                "componentsToAdd": []    
    
    }



yvalue = [0.2, 0.0, -0.2, -0.4, -0.6]
for i in range(len(yvalue)):
    textPosition = { "x":-1.5, 
                 "y": yvalue[i], 
                 "z":zStart}
    markerBall = {
                "type": "Prefabs/tinySphere",
                "parent": parentObject,
                "position": textPosition,
                "scale": {"x":makerSize, "y":makerSize, "z":makerSize},
                "name": "markerBall"+str(i+1), 
                "color": [255, 0, 0, 255],
                "componentsToAdd": []    
    
    }    
    textObjectList.append(markerBall)
    
    
olist = olist #+ textObjectList
#for ow in olist:
#    print(json.dumps(ow, indent=4))
'''
print("DONE!")

DONE!


In [17]:
target = 356
actual = 349

diff = target-actual
diff = (diff + 180) % 360 - 180
print(diff)

7


In [18]:
np.arange(0, 360, 45)

array([  0,  45,  90, 135, 180, 225, 270, 315])

In [19]:
-0.79, -0.01, 1.13

(-0.79, -0.01, 1.13)